In [338]:
#!/usr/bin/env python
#import SciServer.LoginPortal as Login
#token=Login.getToken()
#user = Login.getKeystoneUserWithToken(token)
#import SciServer.CasJobs as CasJobs # query with CasJobs
#import SciServer.SciDrive           # read/write to/from SciDrive
import urllib
from bs4 import BeautifulSoup as bs
#import pandas
import numpy as np
import os
from datetime import datetime
import dateutil
import re
import pandas
#import tabulate
#from scipy import stats
#print(token)

trimmer = re.compile(r'^[ \t]+|[ \t]+$')
capitalized = re.compile('([A-Z][a-zA-Z]*\s*)+')
entryend = re.compile(r'.\s+\[')
oxfordcomma = re.compile(r',\s+(and)')
has_not_number = re.compile(r'\D')

In [339]:
inclusiverange = lambda start, end: range(start, end+1)
datearray = np.empty((24,1),dtype=datetime)
index = 0
for i in inclusiverange(2015,2016):
    for j in inclusiverange(1,12):
        datestring = (str(i)+'-'+str(j)+'-'+'1')
        datearray[index,0] = dateutil.parser.parse(datestring)
        index = index + 1
newdatearray = datearray[0:21,0:1]   # keep up through September 2016
#print(newdatearray)

deadpeoplelist = []
linklist = []
thelist = []
thisday = []
#for i in range(0,2):
for i in range(0,len(newdatearray)):
    pagetitle = "Deaths in "+newdatearray[(i,0)].strftime('%B %Y')
    pagetitle2 = str.replace(pagetitle," ","_")
    pageurl = "http://www.wikipedia.org/wiki/"+pagetitle2
    print(pageurl)
    page = urllib.request.urlopen(pageurl).read()
    soup = bs(page, "html.parser")
    headlines = soup.find_all('span',class_='mw-headline')
    listofdays = []
    headlinecount = 0
    for p in headlines:
        if (headlinecount >= 1 and headlinecount <= len(headlines)-2):
            listofdays.append(int(p.get_text()))
        headlinecount = headlinecount + 1
    ultags = soup.find_all('ul')
    ulcount = 0
    for j in ultags:
        nestedlist = 0
        if ((ulcount >= 2) and (ulcount <= len(listofdays)+1)):
            daynum = ulcount - 1
            for k in range(0,len(j)):
                x = j.contents[k]
                if (x != '\n'):
                    if (x.find('ul') != None):
                        nestedlist = 1
                    else:
                        deadpeoplelist.append(x.get_text())
                        thisday.append(dateutil.parser.parse(newdatearray[(i,0)].strftime('%Y-%-m')+'-'+str(daynum)))
                        y = x.find('a',href=True)
                        linklist.append('https://en.wikipedia.org'+y['href'])
        if (nestedlist == 0):
            ulcount = ulcount + 1
deadnamelist = []
deadagelist = []
for q in deadpeoplelist:
    if (trimmer.search(q)):
        q = re.sub(trimmer,'',q)
    deadnamelist.append(q.split(',')[0])
    try:
        deadagelist.append(int(q.split(',')[1]))
#        print(name,' ',age)
    except:
        deadagelist.append(np.nan)
        scrat = q
#        print(scrat)
#for m in range(0,len(deadpeoplelist)):
#    print(m,' ',deadpeoplelist[m],' ',deadnamelist[m],' ',deadagelist[m])

deadpeople = np.empty((len(deadpeoplelist),8),dtype=object)
deadpeople[:,0] = deadpeoplelist
deadpeople[:,1] = linklist
deadpeople[:,2] = thisday
deadpeople[:,3] = deadnamelist
deadpeople[:,4] = deadagelist
print('Done; last row is ',deadpeople[-1])

http://www.wikipedia.org/wiki/Deaths_in_January_2015
http://www.wikipedia.org/wiki/Deaths_in_February_2015
http://www.wikipedia.org/wiki/Deaths_in_March_2015
http://www.wikipedia.org/wiki/Deaths_in_April_2015
http://www.wikipedia.org/wiki/Deaths_in_May_2015
http://www.wikipedia.org/wiki/Deaths_in_June_2015
http://www.wikipedia.org/wiki/Deaths_in_July_2015
http://www.wikipedia.org/wiki/Deaths_in_August_2015
http://www.wikipedia.org/wiki/Deaths_in_September_2015
http://www.wikipedia.org/wiki/Deaths_in_October_2015
http://www.wikipedia.org/wiki/Deaths_in_November_2015
http://www.wikipedia.org/wiki/Deaths_in_December_2015
http://www.wikipedia.org/wiki/Deaths_in_January_2016
http://www.wikipedia.org/wiki/Deaths_in_February_2016
http://www.wikipedia.org/wiki/Deaths_in_March_2016
http://www.wikipedia.org/wiki/Deaths_in_April_2016
http://www.wikipedia.org/wiki/Deaths_in_May_2016
http://www.wikipedia.org/wiki/Deaths_in_June_2016
http://www.wikipedia.org/wiki/Deaths_in_July_2016
http://www.wikip

In [340]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

# Figure out which row numbers have entries we need to fix
rownumberstofix = []
for i in range(0,len(deadpeople[:,0])):
    if (np.isnan(deadpeople[i,4])):    # using NaN for ages that couldn't be parsed as ints
        rownumberstofix.append(i)

print('Number of entries to fix:',len(rownumberstofix))

#special_case_count = 0   # used to count potential special cases in the loop below

# Check whether the special cases will still need to be fixed.

# iterate through deadpeople, but only those rows we identified as needing fixes above
for k in range(0,len(rownumberstofix)):

    # pick a row number and the raw text entry for that row number
    thisrow = rownumberstofix[k]
    textclue = deadpeople[thisrow,0]
    
    # case 1: junior/senior/III
    if ((textclue.split(',')[1] == ' Jr.') or (textclue.split(',')[1] == ' Sr.') or (textclue.split(',')[1] == ' III')):
        deadpeople[thisrow,3] = deadpeople[thisrow,0].split(',')[0]+','+deadpeople[thisrow,0].split(',')[1]
        deadpeople[thisrow,4] = int(deadpeople[thisrow,0].split(',')[2])

    # case 2: age range with an em-dash
    if (textclue.split(',')[1].find('–') != -1):
        splitspot = textclue.split(',')[1].find('–')
        youngage = int(textclue.split(',')[1][1:splitspot])
        oldage = int(textclue.split(',')[1][splitspot+1:9999])
        avgage = (oldage - youngage)/2
        if (avgage == 1.0):
            agey = youngage + 1   # if age has a 2-year range, pick the middle
        elif (avgage == 0.5):
            agey = np.round(youngage+np.random.rand(),0) # if 1-year, one or the other
        else:
            print('wtf? age text is',textclue.split(',')[1])
        deadpeople[thisrow,4] = int(agey)

    # case 3: circa age
    if (textclue.split(',')[1].find('c. ') != -1):
        splitspot = textclue.split(',')[1].find('c. ')
        agey = textclue.split(',')[1][splitspot+3:9999]
#        print(textclue.split(',')[1])
        deadpeople[thisrow,4] = int(agey)

    # case 4: age followed by question mark
    if (textclue.split(',')[1].find('?') != -1):
        splitspot = textclue.split(',')[1].find('?')
        agey = textclue.split(',')[1][1:splitspot]
#        print(textclue.split(',')[1])
#        print(agey)
        deadpeople[thisrow,4] = int(agey)

    # case 5: stupid nobility with commas in their names
    isbaron = (textclue.split(',')[1].find('Baron') != -1)
    ismarquess = (textclue.split(',')[1].find('Marquess') != -1)
    ismarchoiness = (textclue.split(',')[1].find('Marchioness') != -1)
    isearl = (textclue.split(',')[1].find('Earl') != -1)
    isprince = (textclue.split(',')[1].find('Prince') != -1)
    iscount = (textclue.split(',')[1].find('Count') != -1)
    isviscount = (textclue.split(',')[1].find('Viscount') != -1)
    isduke = (textclue.split(',')[1].find('Duke') != -1)
    isduchess = (textclue.split(',')[1].find('Duchess') != -1)
    isseigneur = (textclue.split(',')[1].find('Seigneur') != -1)
    islord = (textclue.split(',')[1].find('Lord') != -1)
    islady = (textclue.split(',')[1].find('Lady') != -1)
    
    isnoble = (isbaron or ismarquess or ismarchoiness or isearl or isprince or iscount or isviscount or isduke or isduchess or isseigneur or islord or islady)
    if (isnoble):
        #print(textclue.split(',')[1])
        deadpeople[thisrow,3] = textclue.split(',')[0]+','+textclue.split(',')[1]
        deadpeople[thisrow,4] = int(textclue.split(',')[2])

    # case 6: some special case idiots

#    special_case_print_flag = 0
#    if (textclue.split(',')[0] in ('Alexis FitzGerald', 'Syed Rizwan Farook','Tashfeen Malik','Leonidas B. Young','Albert Rhoton','Robbie Brennan','Hafsa Mossi','Jimmy Levine')):
#        print(deadpeople[thisrow,0])
#        special_case_count = special_case_count + 1
#        special_case_print_flag = 1
#    if (textclue.split(',')[0] in ('M. K. Wren','Don Brothwell','Abu Mohammad al-Adnani','Shawshank tree','Neelabh Ashk','Irving Sablosky. 92')):
#        print(deadpeople[thisrow,0])
#        special_case_count = special_case_count + 1
#        special_case_print_flag = 1        
#    if (special_case_print_flag == 1):
#        print("Need to fix",special_case_count,"special case idiots in step 6 below.")

    if (textclue.split(',')[0] == 'Alexis FitzGerald'):
        deadpeople[thisrow,3] = 'Alexis FitzGerald, Jnr'
        deadpeople[thisrow,4] = 70
    if (textclue.split(',')[0] == 'Syed Rizwan Farook'):
        deadpeople[thisrow,4] = 28
    if (textclue.split(',')[0] == 'Tashfeen Malik'):
        deadpeople[thisrow,4] = 29
    if (textclue.split(',')[0] == 'Leonidas B. Young'):
        deadpeople[thisrow,3] = 'Leonidas B. Young, II'
        deadpeople[thisrow,4] = 62
    if (textclue.split(',')[0] == 'Albert Rhoton'):
        deadpeople[thisrow,3] = 'Albert Rhoton, Jr'
        deadpeople[thisrow,4] = 83
    if (textclue.split(',')[0] == 'Robbie Brennan'):
        deadpeople[thisrow,4] = np.round(68+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Hafsa Mossi'):
        deadpeople[thisrow,4] = np.round(51+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Jimmy Levine'):
        deadpeople[thisrow,4] = np.round(61+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'M. K. Wren'):
        deadpeople[thisrow,4] = np.round(77+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Don Brothwell'):
        deadpeople[thisrow,4] = np.round(82+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Abu Mohammad al-Adnani'):
        deadpeople[thisrow,4] = np.round(38+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Shawshank tree'):
        deadpeople[thisrow,4] = 180
    if (textclue.split(',')[0] == 'Neelabh Ashk'):
        deadpeople[thisrow,4] = 70
    if (textclue.split(',')[0] == 'Irving Sablosky. 92'):
        deadpeople[thisrow,3] = 'Irving Sablosky'
        deadpeople[thisrow,4] = 92

# Done! What still needs to be fixed? At this point, should be just those with unreported ages.
rownumbersstilltofix = []
for i in range(0,len(deadpeople[:,0])):
    if (np.isnan(deadpeople[i,4])):
        rownumbersstilltofix.append(i)
        
print('Number of unreported ages remaining after fixes:',len(rownumbersstilltofix))

# And lastly, print out those entries with unreported ages (now stored as -9999)
#for i in range(0,len(rownumbersstilltofix)):
#    thatrow = rownumbersstilltofix[i]
#    print(deadpeople[thatrow,0],':',deadpeople[thatrow,3])

Number of entries to fix: 247
Number of unreported ages remaining after fixes: 49


In [341]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

print('Total entries:',len(deadpeople))

mask = np.ones(len(deadpeople),dtype=bool)
#print(mask)

keywords = []
keywords.append(re.compile(r'cat\b'))
keywords.append(re.compile(r'frog\b'))
keywords.append(re.compile(r'dog\b'))
keywords.append(re.compile(r'horse\b'))
keywords.append(re.compile(r'tree\b'))
keywords.append(re.compile(r'parrot\b'))

#keywords = [' frog',' cat']
for i in range(0,len(deadpeople)):
    for j in range(0,len(keywords)):
        if(keywords[j].search(deadpeople[i,0])):
            mask[i] = 0

#Some exceptions to fix!
for k in range(0,len(deadpeople)):
    if (deadpeople[k,3] == 'George Attla'):  # dog musher
        mask[k] = 1
    if (deadpeople[k,3] in ('H. Allen Jerkens','Jim Mitchell','Sir Peter O\'Sullevan','John A. Nerud','Bernard Secly','Wallace Dollase','Jack Brizendine','Alan Lee','Sheila Varian','Ogden Mills Phipps','Charles Brantley','Raymond Mould')): # horse breeders, trainers, etc.
        mask[k] = 1
    if (deadpeople[k,3] == 'Khalid Mahmood Mithu'): # killed by falling tree
        mask[k] = 1
    if (deadpeople[k,3] in ('André Turcat','Antonella Steni')): # name or description contains "cat"
        mask[k] = 1

number_masked = 0
for i in range(0,len(mask)):
    if (mask[i] == False):
        number_masked = number_masked + 1
print('Animals masked out:',number_masked)

deadpeople = deadpeople[mask]

print('Remaining entries:',len(deadpeople))

#print('\n')
#print('to be masked out:')
#for i in range(0,len(deadpeople)):
#    if (mask[i]):
#        print(deadpeople[i,0])

#print('\n')
#print('staying in:')
#for i in range(0,50):       
#for i in range(0,len(deadpeople)):
#    if (not(mask[i])):
#        print(deadpeople[i,0])

Total entries: 11228
Animals masked out: 58
Remaining entries: 11170


In [374]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

# w00t, you can write SQL-like statements!
# The equivalent of "select * from deadpeople where age != -9999" is:
#### deadpeople[deadpeople[:,4] != -9999]

#print(deadpeople)
nationality = []

for k in range(0,len(deadpeople)):
    # pick a row number and the raw text entry for that row number
    textclue = re.sub(trimmer,'',deadpeople[k,0])

#    if (k in (0,2127,7619,8664)):
#        print(k,':',deadpeople[k,3],':',deadpeople[k,4],':',textclue)

#    quicksummary = textclue.split(',')[2]
    namelength = len(deadpeople[k,3])

    if (np.isnan(deadpeople[k,4])):
        agelength = 0
    else:
        if re.search(r'\D',textclue.split(',')[1]):
            agelength = len(textclue.split(',')[1]) - 1
        else:
            agelength = len(str(deadpeople[k,4]))

            
    quicksummary = textclue[namelength+agelength+4:99999]
    
    if (trimmer.search(quicksummary)):      # find leading and trailing whitespace
        quicksummary = re.sub(trimmer,'',quicksummary)        # if found, trim    
        
    nationality.append('')
    if (capitalized.search(quicksummary.split(' ')[0])):
        nationality[k] = quicksummary.split(' ')[0]
    country2words = False
    # Fix nationalities with 2 words: "New" and "South" etc.
    if (nationality[k] in ('New','South','North','East','Sri','Puerto','Costa','Hong','Sierra','Cape','Cook','El','Saint','Central','Northern')):
        nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]
        country2words = True

# special case 3 words: Northern Mariana Islands, Papua New Guinea
    if (nationality[k] == 'Northern Mariana'):
        nationality[k] = nationality[k]+' '+quicksummary.split(' ')[2]
        
    if (nationality[k] == 'Papua'):
        nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]+' '+quicksummary.split(' ')[2]

# special case: Saudi vs. Saudi Arabian / Dominican vs. Dominican Republic
    if (nationality[k] == 'Saudi'):
        if (quicksummary.split(' ')[1].find('Arabian') != -1):
            nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]
    if (nationality[k] == 'Dominican'):
        if (quicksummary.split(' ')[1].find('Republic') != -1):
            nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]

# Special case: Former Yugoslavia and Soviet Union
    yugos = ['Serbian','Serb','Croatian','Bosnian','Slovenian']
    if (nationality[k] in yugos):
        if (quicksummary.split(' ')[1].find('Yugoslav') != -1):
            nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]
    soviets = ['Russian','Ukrainian','Kyrgyz','Belarusian','Azerbaijani']
    if (nationality[k] in soviets):
        if (quicksummary.split(' ')[1].find('Soviet') != -1):
            nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]
    if (nationality[k] == 'Soviet'):
        if (quicksummary.find('Russian') != -1):
            nationality[k] = 'Soviet Russian'
    
    # Fix 'X-born Y' nationalities
    if (nationality[k].find('-born') != -1):
        if (country2words):
            if (capitalized.search(quicksummary.split(' ')[2])):
                nationality[k] = nationality[k]+' '+quicksummary.split(' ')[2]
        else:
            if (capitalized.search(quicksummary.split(' ')[1])):
                nationality[k] = nationality[k]+' '+quicksummary.split(' ')[1]
    # Special case: New Zealand citizens born abroad
    if (nationality[k].find('-born New') != -1):
        nationality[k] = nationality[k] + ' Zealand'
    # Special case: South Afrian citizens born abroad
    if (nationality[k].find('-born South') != -1):
        nationality[k] = nationality[k] + ' African'
        
    # Special case: Frederick Bruce-Lyle, 62, Ghanaian-born Saint Vincentian judge
    if (nationality[k].find('-born Saint') != -1):
        nationality[k] = 'Ghanaian-born Saint Vincentian'
    # special case: Carl Thompson, 33, Heavy person from Britain
    if (nationality[k] == 'Heavy'):
        nationality[k] = 'British'
    # special case: Sergio Adolfo Govi, 81, Italian-born Central African Roman Catholic prelate
    if (nationality[k] == 'Italian-born Central'):
        nationality[k] = 'Italian-born Central African'
    # special case: Sir Roger Lobo, 91, Macanese-born Hong Kong businessman and politician
    if (nationality[k] == 'Macanese-born Hong'):
        nationality[k] = 'Macanese-born Hong Kong'
    # special case: Tino García, 80, Nicaraguan-Puerto Rican actor
    if (nationality[k] == 'Nicaraguan-Puerto'):
        nationality[k] = 'Nicaraguan-Puerto Rican'
    # special case: Ziad Khalaf Raja al-Karbouly, Iraqi Al-Qaeda officer
    if (nationality[k] == 'l-Qaeda'):
        nationality[k] = 'Iraqi'
    # special case: 'Marjorie Elliott Sypher, 89, Canadian-born Costa Rican First Lady
    if (nationality[k] == 'Canadian-born Costa'):
        nationality[k] = 'Canadian-born Costa Rican'
    # special case: Sir John Yocklunn, 82, Australian-Papua New Guinean
    if (nationality[k] == 'Australian–Papua'):
        nationality[k] = 'Australian–Papua New Guinean'
    # special case: Polish-born Holocaust survivor
    if (nationality[k] == 'Polish-born Holocaust'):
        nationality[k] = 'Polish-born'
        
    # trim for good luck
    if (trimmer.search(nationality[k])):
        nationality[k] = re.sub(trimmer,'',nationality[k])

deadpeople[:,5] = nationality

#print(deadpeople)
# checking through all values

x = pandas.DataFrame(deadpeople[:,[0,3,5]],columns=['text','name','nationality'])
x.sort_values('nationality')
#x['nationality'] = '/'+x['nationality']+'/'

y = x.groupby('nationality').size()
#y[y < 7].sort_values(ascending=False)
#y[(y == 1)][300:350]  # this shows first 50 values with a certain cout
# special case rows to fix: 



nationality
Ukrainian-born Canadian     1
Ukrainian-born French       1
Ukrainian-born Polish       1
Uruguayan-Italian           1
Uzbek-born Russian          1
Uzbekistani                 1
Venezuelan-born Canadian    1
Venezuelan-born French      1
Vietnamese-French           1
Vietnamese-born French      1
Welsh-American              1
Western                     1
ni-Vanuatu                  1
dtype: int64

In [397]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)
##  5: nationality (parsed)

claim = []

for k in range(0,len(deadpeople)):
    textclue = deadpeople[k,0]

    namelength = len(re.sub(trimmer,'',deadpeople[k,3]))

    if (np.isnan(deadpeople[k,4])):
        agelength = 0
    else:
        agelength = len(str(deadpeople[k,4]))

    nationalitylength = len(re.sub(trimmer,'',deadpeople[k,5]))
    remainder = textclue[namelength+agelength+nationalitylength+4:99999]
    
    if (trimmer.search(remainder)):               # find leading and trailing whitespace
        remainder = re.sub(trimmer,'',remainder)        # if found, trim
        
    claim.append('')
    
# entries usually end with a period and a bracketed number. Remove this part.
    if (remainder.find('.[') != -1):
        remainder = remainder[0:remainder.find('.[')] + '. ['
    if (entryend.search(remainder)):
        remainder = remainder[0:re.search(entryend,remainder).start()]

# parens provide more information on the main claim to fame. So trim off parens.
    if (remainder.find('(') != -1):
        # BUT closed parens then 'and' can indicate multiple claims to fame.
        if (re.search(parens_then_and,remainder)):
            # if so, just snip out the part in parens
            remainder = remainder[0:remainder.find('(')-1]+remainder[remainder.find(')')+1:len(remainder)]
            # one more time in case there are 2 instances of paren content
            if (remainder.find('(') != -1):
                remainder = remainder[0:remainder.find('(')-1]
        else:
            # otherwise, snip behind left paren
            remainder = remainder[0:remainder.find('(')-1]

# snip off just behind the first comma... except...
### an Oxford commma (,...aned) means we need to keep past the comma 
##### (except for cause of death, come back to that)
    if ((remainder.find(',') != -1) and (not(re.search(oxfordcomma,remainder)))):
        remainder = remainder.split(',')[0]

#    if (remainder == '2, Nigerian diplomat, Ambassador to the Soviet Union, Netherlands, Cameroon, the Bahamas, and the United States'):
#        print(k,remainder)
    claim[k] = remainder
#    stillremaining = textclue[namelength+agelength+nationalitylength+claimlength+6:99999]    
#        print(k,':',claim[k],'+',stillremaining)

deadpeople[:,6] = claim

#print(deadpeople[:,[0,3,5,6]])


# special case I can't be arsed to figure out how to handle
deadpeople[deadpeople[:,3] == 'Steve Nave',5] = 'American'
deadpeople[deadpeople[:,3] == 'Steve Nave',6] = 'actor and casting director'

deadpeople[deadpeople[:,3] == 'Víctor Santiago Díaz',5] = 'Puerto Rican'
deadpeople[deadpeople[:,3] == 'Víctor Santiago Díaz',6] = 'politician'

deadpeople[deadpeople[:,3] == 'Mohamed Olow Barrow',5] = 'Somali'
deadpeople[deadpeople[:,3] == 'Mohamed Olow Barrow',6] = 'politician'

deadpeople[deadpeople[:,3] == 'Muhayadin Mohamed',5] = 'Somali'
deadpeople[deadpeople[:,3] == 'Muhayadin Mohamed',6] = 'politician'

#print(deadpeople[deadpeople[:,3] == 'Steve Nave'],5)

x = pandas.DataFrame(deadpeople[:,[0,3,5,6]],columns=['text','name','nationality','claim'])

x.sort_values('claim')
#x[x['claim'] == '2, Nigerian diplomat, Ambassador to the Soviet Union, Netherlands, Cameroon, the Bahamas, and the United States']

# checking through all values
y = x.groupby('claim').size()
#y[y == 4]#.sort_values(ascending=False)
y[(y == 2)][150:200]  # this shows first 50 values with a certain cout


# special case rows to fix: 

#print(x[(x['nationality'] == 'P')])
#print(x[(x['nationality'] == '(A')])
#print(x[(x['nationality'] == 'Dominican Republic-born Republic-born')])
#print(x[(x['nationality'] == 'Chancellor')])
#print(x[(x['nationality'] == 'Seigneur')])

#print(x[(x['claim'] == 'aqi militant')])




claim
environmentalist                    2
fantasy author                      2
farmer and politician               2
fashion model                       2
film and television actress         2
film director and actor             2
film score composer                 2
flamenco singer                     2
flautist                            2
flyweight boxer                     2
folk singer-songwriter              2
football executive                  2
football player and politician      2
footballer and politician           2
furniture designer                  2
geophysicist                        2
golf player                         2
gospel musician                     2
harpsichordist                      2
heavyweight boxer                   2
herbalist                           2
hip hop musician                    2
hip-hop musician                    2
historian and archaeologist         2
historian and educator              2
horticulturalist                    2
hoteli

In [ ]:
causeofdeath = []

commoncauses = []

# be careful with these
commoncauses.append('fall')
commoncauses.append('tumo')
commoncauses.append('fire')

# common natural causes
commoncauses.append('heart attack')
commoncauses.append('stroke')
commoncauses.append('diabetes')
commoncauses.append('cancer')
commoncauses.append('dengue')
commoncauses.append('cardiac arrest')
commoncauses.append('lymphoma')
commoncauses.append('embolism')
commoncauses.append('pneumonia')
commoncauses.append('bleeding')
commoncauses.append('leuk')  #leukemia/leukaemia
commoncauses.append('sclerosis')
commoncauses.append('dementia')
commoncauses.append('malaria')
commoncauses.append('emphysema')
commoncauses.append('natural causes')
commoncauses.append('seizure')

# violent causes
commoncauses.append('suicide')
commoncauses.append('burned')
commoncauses.append('airstrike')
commoncauses.append('air strike')
commoncauses.append('shot')
commoncauses.append('executed')
commoncauses.append('behead')
commoncauses.append('collision')
commoncauses.append('crash')
commoncauses.append('hanging')
commoncauses.append('stabbed')
commoncauses.append('drowned')
commoncauses.append('murdered')
commoncauses.append('drone')
commoncauses.append('hit by car')
commoncauses.append('severed')
commoncauses.append('earthquake')
commoncauses.append('asphyxiation')
commoncauses.append('euthanasia')
commoncauses.append('overdose')
commoncauses.append('intoxication')
commoncauses.append('stampede')
commoncauses.append('beaten')
commoncauses.append('homicide')
commoncauses.append('strangled')
commoncauses.append('explosion')
commoncauses.append('electrocution')
commoncauses.append('building collapse')
commoncauses.append('struck by')
commoncauses.append('gored')
commoncauses.append('bomb')

# less common natural causes
commoncauses.append('mesothelioma')
commoncauses.append('septic shock')
commoncauses.append('progressive supranuclear palsy')
commoncauses.append('aneur')  #aneurysm/#aneurism
commoncauses.append('morrha') #haemorrhage/hemorrhage
commoncauses.append('myeloma')
commoncauses.append('fibrosis')
commoncauses.append('edema')
commoncauses.append('hypothermia')
commoncauses.append('blood clot')
commoncauses.append('melanoma')
commoncauses.append('enceph') #encephalitis
commoncauses.append('infect') 
commoncauses.append('aortic dissection')
commoncauses.append('ALS')
commoncauses.append('hematoma')
commoncauses.append('pancreatitis')
commoncauses.append('sepsis')
commoncauses.append('sarcoma')
commoncauses.append('asthma')
commoncauses.append('pulmonary distress')
commoncauses.append('respiratory')
commoncauses.append('cerebral')
commoncauses.append('bronchospasm')
commoncauses.append('COPD')
commoncauses.append('jaundice')

# words indicating death but not specifically
commoncauses.append('disease')
commoncauses.append('syndrome')
commoncauses.append('complication')
commoncauses.append('illness')
commoncauses.append('ailment')
commoncauses.append('failure')
commoncauses.append('disorder')
commoncauses.append('killed')
commoncauses.append('injur') # injury/injuries
commoncauses.append('incident')
commoncauses.append('atroph')
commoncauses.append('accident')

for k in range(0,len(deadpeople)):
        claimlength = len(claim[k])
    remainingsummary = textclue[namelength+agelength+nationalitylength+claimlength+5:99999]

    causeofdeath.append('')
                
    if (remainingsummary.find(",") != -1):
        thiscause = remainingsummary.split(',')[1]
#        causeofdeath[k] = remainingsummary.split(',')[1]
    if (thiscause.find("[") != -1):
        thiscause = thiscause[0:thiscause.find('[')-1]
#        causeofdeath[k] = causeofdeath[k][0:causeofdeath[k].find('[')-1]

#    not_common_death_flag = 1
    for i in range(0,len(commoncauses)):
        if (thiscause.find(commoncauses[i]) != -1):
            causeofdeath[k] = thiscause
#            not_common_death_flag = 0

#    if ((not_common_death_flag == 1) and (causeofdeath[k] != '\n') and (k > 10700)):
#        print(k,':',causeofdeath[k])
#print(causeofdeath) 
#print(stats.itemfreq(causeofdeath))

deadpeople[:,7] = causeofdeath
#print(pandas.value_counts(causeofdeath))

#deadframe = pandas.DataFrame(data=deadpeople[0:,0:])
#print(deadframe[7][deadframe[7] == ' convicted of financing the 1993 Bombay bombings'])

print(deadpeople[deadpeople[:,7] == ' convicted of financing the 1993 Bombay bombings'])
#pd.DataFrame(data=data[1:,1:],    # values
#...              index=data[1:,0],    # 1st column as index
#...              columns=data[0,1:])  # 1st row as the column names


#convicted of financing the 1993 Bombay bombings
#survivor of National Airlines Flight 101 crash

# The equivalent of "select * from deadpeople where age != -9999" is:
#### deadpeople[deadpeople[:,4] != -9999]
#print(deadpeople)